# Here is my attempt to write a model based on an old Kaggle Cats vs Dog dataset. The dataset can be seen at the links shown and has 25,000 images of cats and dogs.
https://www.kaggle.com/c/dogs-vs-cats/data 

In [118]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../koyel/input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test
train



In [120]:
train_path = "../koyel/input/train"
test_path = "../koyel/input/test"

ROWS = 64
COLS = 64
CHANNELS = 3

images      = [img for img in os.listdir(train_path)]
images_dog  = [img for img in os.listdir(train_path) if "dog" in img]
images_cat  = [img for img in os.listdir(train_path) if "cat" in img]

images_test = [img for img in os.listdir(test_path)]

In [121]:
#only taking a subset (less accuracy but faster training)
train_dog = images_dog[:1000]
train_cat = images_cat[:1000]
valid_dog = images_dog[1000:1100]
valid_cat = images_cat[1000:1100]

train_list = train_dog + train_cat
valid_list = valid_dog + valid_cat
test_list  = images_test[0:]

shuffle(train_list)

train = np.ndarray(shape=(len(train_list),ROWS, COLS))
train_color = np.ndarray(shape=(len(train_list), ROWS, COLS, CHANNELS), dtype=np.uint8)
test = np.ndarray(shape=(len(test_list),ROWS, COLS))
test_color = np.ndarray(shape=(len(images_test), ROWS, COLS, CHANNELS), dtype=np.uint8)
valid = np.ndarray(shape=(len(valid_list), ROWS, COLS))
valid_color = np.ndarray(shape=(len(valid_list), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [122]:
labels = np.ndarray(len(train_list))

for i, img_path in enumerate(train_list):
    img_color = cv2.imread(os.path.join(train_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    train[i] = img
    train_color[i] = img_color
   
    if "dog" in img_path:
        labels[i] = 0
    else:
        labels[i] = 1

In [123]:
valid_labels = np.ndarray(len(valid_list))

for i, img_path in enumerate(valid_list):
    img_color = cv2.imread(os.path.join(train_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    valid[i] = img
    valid_color[i] = img_color
    
    if "dog" in img_path:
        valid_labels[i] = 0
    else:
        valid_labels[i] = 1

In [124]:
for i, img_path in enumerate(test_list):
    img_color = cv2.imread(os.path.join(test_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    test[i] = img
    test_color[i] = img_color

In [125]:
for i, img_path in enumerate(test_list):
    img_color = cv2.imread(os.path.join(test_path, img_path), 1)
    img_color = cv2.resize(img_color, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    test[i] = img
    test_color[i] = img_color



In [126]:
from keras.utils import np_utils

X_train = train_color / 255
X_valid = valid_color / 255
X_test  = test_color  / 255
# one hot encode outputs
y_train = np_utils.to_categorical(labels)
y_valid = np_utils.to_categorical(valid_labels)
num_classes = y_valid.shape[1]

In [127]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(30, 5, 5, border_mode='valid', input_shape=(64, 64, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [131]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), nb_epoch=50, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_valid, y_valid, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

C:\Users\koyel\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, (5, 5), padding="valid", activation="relu", input_shape=(64, 64, 3...)`
C:\Users\koyel\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, (3, 3), activation="relu")`
C:\Users\koyel\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2000 samples, validate on 200 samples
Epoch 1/50
17s - loss: 0.6880 - acc: 0.5305 - val_loss: 0.6951 - val_acc: 0.5300
Epoch 2/50
15s - loss: 0.6550 - acc: 0.5965 - val_loss: 0.6753 - val_acc: 0.5950
Epoch 3/50
15s - loss: 0.6291 - acc: 0.6350 - val_loss: 0.6654 - val_acc: 0.6100
Epoch 4/50
15s - loss: 0.5949 - acc: 0.6810 - val_loss: 0.6738 - val_acc: 0.6000
Epoch 5/50
16s - loss: 0.5766 - acc: 0.6895 - val_loss: 0.6506 - val_acc: 0.6600
Epoch 6/50
15s - loss: 0.5469 - acc: 0.7250 - val_loss: 0.6362 - val_acc: 0.6750
Epoch 7/50
15s - loss: 0.5470 - acc: 0.7120 - val_loss: 0.6346 - val_acc: 0.6600
Epoch 8/50
17s - loss: 0.4941 - acc: 0.7510 - val_loss: 0.6497 - val_acc: 0.6650
Epoch 9/50
16s - loss: 0.4818 - acc: 0.7645 - val_loss: 0.6636 - val_acc: 0.6500
Epoch 10/50
16s - loss: 0.4712 - acc: 0.7720 - val_loss: 0.6523 - val_acc: 0.6550
Epoch 11/50
16s - loss: 0.4228 - acc: 0.8075 - val_loss: 0.6619 - val_acc: 0.6600
Epoch 12/50
18s - loss: 0.3935 - acc: 0.8200 - val_loss: 0.7

In [132]:
submission = model.predict_classes(X_test, verbose=2)

In [133]:
pd.DataFrame({"id": list(range(1,len(test_color)+1)), 
              "label": submission}).to_csv('submission.csv', index=False,header=True)